In [ ]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

from utils.reproduce_functions import *
from utils.mutation_functions import *
from utils.parent_selection_functions import *
from utils.other_fucntions import *

import lab9_lib

In [ ]:
PROBLEM_SIZE = 5

MU = 15
LAMBDA = 30
strategy = 'plus' # comma or plus

MUTATION_PROB = 0.2
DYNAMIC_MUTATION_PROB = True
DIVERSITY_THRESHOLD = 20

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 3_000
COOLDOWN_TIME = 100

In [ ]:
mutate = three_bit_flip
reproduce = uniform_crossover
parent_selection = roulette

In [ ]:
def ga(fitness, parents, parents_evals, memoization=False, history=[]):
    generation_convergence = -1

    if memoization:
        pop_history = {}
        for i in range(len(parents)):
            pop_history[parents[i].tobytes()] = parents_evals[i]

    best_value = -1 
    last_change = -1
    
    for generation in tqdm(range(NUMBER_GENERATIONS)):
        offsprings = []
        offsprings_evals = []
        while len(offsprings) < LAMBDA:
            # Parent Selection
            p1, p2 = parent_selection(parents, parents_evals)

            # Reproduce Parents
            off_spring = reproduce(p1, p2)

            # Mutate Offspring
            if DYNAMIC_MUTATION_PROB:
                p_div = get_parents_diversity(p1, p2)
                new_ind1 = mutate(off_spring, \
                                  mutation_probability=(1 - (min(p_div,LENGTH_SOLUTION/2)/(LENGTH_SOLUTION/2))) )
            else:    
                new_ind1 = mutate(off_spring, mutation_probability=MUTATION_PROB)

            # Evaluate Offspring
            if memoization:
                new_ind1_eval = pop_history.get(new_ind1.tobytes())
                if new_ind1_eval == None:
                    new_ind1_eval = fitness(new_ind1)
                    pop_history[new_ind1.tobytes()] = new_ind1_eval
            else:
                new_ind1_eval = fitness(new_ind1)

            # Add it to new population
            offsprings.append(new_ind1)
            offsprings_evals.append(new_ind1_eval)

        # Create new population (with parents if plus, without if comma)
        all_people = (parents if strategy=='plus' else []) + offsprings
        all_evals = (parents_evals if strategy=='plus' else []) + offsprings_evals
        best_people = np.argsort(all_evals)[::-1]

        best_eval = all_evals[best_people[0]]
        history.append(best_eval)
        if best_eval > best_value:
            best_value = best_eval
            last_change = generation
        elif generation - last_change > COOLDOWN_TIME:
            break

        parents = []
        parents_evals = []
        for i in range(MU):
            parents.append(all_people[best_people[i]])
            parents_evals.append(all_evals[best_people[i]])
            
        if np.max(parents_evals) - 1.0 >= 0 and generation_convergence == -1:
            generation_convergence = generation, fitness.calls

    return parents, parents_evals, generation_convergence

Without memoization

In [ ]:
'''
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
history = []
parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(MU)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals, gc = ga(fitness, parents, parents_evals, memoization=False, history=history)
i_best = np.argmax(parents_evals)
# print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)
print("Generation, NumCalls @ fitness=1.0 : ", gc)
'''

In [ ]:
# fitness = lab9_lib.make_problem(PROBLEM_SIZE)
# history = []
# parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(MU)]
# parents_evals = [fitness(x) for x in parents]
# parents, parents_evals, gc = ga(fitness, parents, parents_evals, memoization=False, history=history)
# i_best = np.argmax(parents_evals)
# # print(parents[i_best])
# print("Best score: ", parents_evals[i_best])
# print("Num fitness calls: ", fitness.calls)
# print("Generation, NumCalls @ fitness=1.0 : ", gc)

#import matplotlib.pyplot as plt

#plt.plot(history)
#plt.xlabel('Generation')
#plt.ylabel('Fitness')
#plt.show()

With memoization

In [ ]:
fitness_calls = []
best_scores = []
for i in range(10):
    fitness = lab9_lib.make_problem(PROBLEM_SIZE)
    history = []
    parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50)]
    parents_evals = [fitness(x) for x in parents]
    parents, parents_evals, gc = ga(fitness, parents, parents_evals, memoization=True, history=history)
    #i_best = np.argmax(history)
    # print(parents[i_best])
    #print("Best score: ", np.max(history))
    #print("Num fitness calls: ", fitness.calls)
    #print("Generation, NumCalls @ fitness=1.0 : ", gc)
    fitness_calls.append(fitness.calls)
    best_scores.append(np.max(history))

print("average fitness calls: ", np.mean(fitness_calls))
print("average best score: ", np.mean(best_scores))


In [ ]:
# plot history

# plot history
import matplotlib.pyplot as plt

plt.plot(history)
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.show()